In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_vertical_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,3357.56,-0.01,0.00,-0.01,-0.01,-0.00,-0.00,0.00
3,-0.01,923.11,-0.00,0.01,0.00,0.00,0.00,-0.00
4,0.00,-0.00,209.25,-0.00,-0.00,0.00,-0.00,0.00
5,-0.01,0.01,-0.00,47.52,0.00,0.00,0.00,-0.00
6,-0.01,0.00,-0.00,0.00,10.08,-0.00,0.00,0.00
7,-0.00,0.00,0.00,0.00,-0.00,2.03,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.45,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00000,-0.00002,-0.00004,-0.00001,-0.00004,0.00001
3,-0.00001,1.00000,-0.00000,0.00003,0.00001,0.00003,0.00000,-0.00003
4,0.00000,-0.00000,1.00000,-0.00004,-0.00002,0.00006,-0.00002,0.00002
5,-0.00002,0.00003,-0.00004,1.00000,0.00002,0.00000,0.00002,-0.00004
6,-0.00004,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00003,0.00006
7,-0.00001,0.00003,0.00006,0.00000,-0.00001,1.00000,-0.00002,-0.00002
8,-0.00004,0.00000,-0.00002,0.00002,0.00003,-0.00002,1.00000,0.00002
9,0.00001,-0.00003,0.00002,-0.00004,0.00006,-0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.02872375997727555

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.41161533e+06 1.04336005e+05 5.21600152e+03 2.58231253e+02
 1.09103622e+01 3.96813368e-01 1.55085158e-02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999062,-0.042349,-0.008808,-0.001995,-0.000425,-0.000086,-0.000019,-0.000005
1,0.041944,0.998275,-0.040135,-0.008586,-0.001817,-0.000365,-0.000081,-0.000022
2,0.010362,0.039255,0.997967,-0.048135,-0.009609,-0.001924,-0.000431,-0.000116
3,0.002816,0.010247,0.047107,0.997213,-0.055806,-0.010641,-0.002361,-0.000643
4,0.000746,0.002702,0.011954,0.054349,0.995858,-0.070193,-0.014805,-0.004002
5,0.000197,0.000712,0.003143,0.013879,0.067178,0.990471,-0.115738,-0.029252
6,0.000061,0.000220,0.000971,0.004273,0.020165,0.102663,0.958154,-0.266411
7,0.000035,0.000128,0.000564,0.002489,0.011714,0.058164,0.261381,0.963407


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0009380291263807461,
 0.0017251279484293836,
 0.0020328303448035046,
 0.002786650722278572,
 0.004141993292145596,
 0.009528851260470206,
 0.041845771604240745,
 0.03659304935691077]